---
layout: post
title: API do IBGE
subtitle: Exercícios e Referências
tags: [python, pycharm, jupyter, package, ibge, api]
image: /img/posts/pandas_icon.png
bigimg: /img/posts/pandas_big.png
gh-repo: michelmetran/api_ibge
gh-badge: [follow, star, watch, fork]
comments: true

---

# SABESP

In [ ]:
import os
import time
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

In [ ]:
# Escolhe o driver Firefox com Profile e Options

driver = webdriver.FirefoxProfile()
driver.set_preference('intl.accept_languages', 'pt-BR, pt')
#driver.set_preference('browser.download.folderList', '2')
#driver.set_preference('browser.download.manager.showWhenStarting', 'false')
#driver.set_preference('browser.download.dir', 'dwnd_path')
#driver.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/octet-stream,application/vnd.ms-excel')

options = Options()
options.headless = False

driver = webdriver.Firefox(firefox_profile=driver, options=options)

In [ ]:
# Cria um driver

site = 'http://mananciais.sabesp.com.br/HistoricoSistemas'
driver.get(site)

## Escolhe o Sistema

In [ ]:
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').click()

In [ ]:
sistema = {0: 'Cantareira',
           1: 'Alto Tietê',
           2: 'Guarapiranga',
           3: 'Cotia',
           4: 'Rio Grande',
           5: 'Rio Claro',
           6: 'São Lourenço',}

In [ ]:
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').send_keys(sistema[0])
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').send_keys(sistema[0])
# ToDo: Check URL

## Cria o Objeto Soup

In [ ]:
WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID,"contenttabledivjqxGrid")))

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
print(soup.prettify())

## Vê e Escolhe a Data

## Cabeçalho

In [ ]:
# Cabeçalho
header = soup.find_all('div', {'class': 'jqx-grid-column-header'})
for i in header:
    print(i.get_text())

In [ ]:
# Seleciona as relevantes
head = []
for i in header:
    if i.get_text().startswith(('Represa', 'Equivalente')):
        print('Excluído: ' + i.get_text())
    else:
        print(i.get_text())
        head.append(i.get_text())
    
print('-'*70)
print(head)
print('-'*70)
print('Número de Colunas: ' + str(len(head)))

## Valores

In [ ]:
# Valores
data = soup.find_all('div', {'class': 'jqx-grid-cell'})
values = []
for i in data:
    #print(i.get_text())
    values.append(i.get_text())
    
#print(values)

## Monta Tabela

In [ ]:
import numpy as np
import pandas as pd

# Convert data to numpy array
num = np.array(values)

# Currently its shape is single dimensional
n_rows = int(len(num)/len(head))
n_cols = int(len(head))
reshaped = num.reshape(n_rows, n_cols)

# Construct Table
pd.DataFrame(reshaped, columns=head)

## Vê e Escolhe a Data

In [ ]:
# Encontra a Data Selecionada

data = soup.find('span', id='lblDataSelecionada')
data_sel = data.text
data_sel

In [ ]:
# Seleciona o Campo para escolher uma nova data

from selenium.webdriver.common.by import By

senddata = driver.find_element_by_xpath('//span[@id="lblDataSelecionada"]')
senddata
senddata.click()

# Return X months
for i in range(4):
    driver.find_element(By.XPATH, '//button[@class="pika-prev"]').click()
    
driver.find_element(By.XPATH, '//button[@class="pika-button pika-day"]').click()

## Encerra o Navegador

In [ ]:
driver.quit()